In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessing/train.csv
/kaggle/input/preprocessing/test.csv


In [4]:
!pip install afinn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 708.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53429 sha256=fd798c1f656c13375bd362b9ce40111fcf511a2f093b89340fd0e8b04c6e51ac
  Stored in directory: /root/.cache/pip/wheels/b0/05/90/43f79196199a138fb486902fceca30a2d1b5228e6d2db8eb90
Successfully built afinn


In [6]:
# Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import time
from sklearn.preprocessing import LabelEncoder
from afinn import Afinn

# Initialize AFINN
afinn = Afinn()

# Download necessary NLTK data
nltk.download('stopwords')

# Data Preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    stemmer = SnowballStemmer('english')
    tokens = text.split()
    filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stopwords.words('english')]
    return ' '.join(filtered_tokens)


print("hello1")

# Assuming 'data.csv' is your dataset file with columns ['review', 'label']
train_df = pd.read_csv('/kaggle/input/preprocessing/train.csv')
test_df = pd.read_csv('/kaggle/input/preprocessing/test.csv')



# Preprocess reviews
train_df['processed_review'] = train_df['review'].apply(preprocess_text)
test_df['processed_review'] = test_df['review'].apply(preprocess_text)



# Function to apply AFINN scores in batches
def apply_afinn_in_batches(dataframe, batch_size=1):
    scores = []  # List to hold the scores
    total_batches = len(dataframe) // batch_size + (1 if len(dataframe) % batch_size else 0)
    
    for batch_num in range(total_batches):
        start_index = batch_num * batch_size
        end_index = start_index + batch_size
        batch = dataframe['processed_review'][start_index:end_index]
        
        # Apply AFINN score to the current batch
        batch_scores = batch.apply(lambda review: afinn.score(review))
        scores.extend(batch_scores)
        
        print(f"Batch {batch_num + 1}/{total_batches} processed.")
    
    return scores

# Apply AFINN in batches to both train and test sets
train_df['afinn_score'] = apply_afinn_in_batches(train_df)
test_df['afinn_score'] = apply_afinn_in_batches(test_df)

print("hello1")
# The rest of your code for preparing features and labels for the training set
X_train = train_df['processed_review']
y_train = train_df['sentiment']
X_test = test_df['processed_review']
y_test = test_df['sentiment']

# Encode the labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['sentiment'])
y_test = label_encoder.transform(test_df['sentiment'])

# Vectorize the text
tfidf_vectorizer = TfidfVectorizer(max_features=50000, ngram_range=(1,3))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Add AFINN scores as a feature
X_train_afinn = train_df['afinn_score'].values.reshape(-1, 1)
X_test_afinn = test_df['afinn_score'].values.reshape(-1, 1)

# Combine TF-IDF features with AFINN scores
X_train_combined = np.hstack((X_train_tfidf.toarray(), X_train_afinn))
X_test_combined = np.hstack((X_test_tfidf.toarray(), X_test_afinn))

# Start timing for prediction
start_predict_time = time.time()

# Predict on the test set
predictions = model.predict(X_test_combined)

# End timing for prediction
end_predict_time = time.time()

# Calculate the elapsed time for prediction
predict_elapsed_time = end_predict_time - start_predict_time
print(f"Time taken for prediction: {predict_elapsed_time:.4f} seconds")

# Evaluate and print the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy on test set:", accuracy)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
hello1


KeyboardInterrupt: 

In [9]:
# Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Download necessary NLTK data
nltk.download('stopwords')

# Data Preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    stemmer = SnowballStemmer('english')
    tokens = text.split()
    filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stopwords.words('english')]
    return ' '.join(filtered_tokens)

# Assuming 'data.csv' is your dataset file with columns ['review', 'label']
train_df = pd.read_csv('/kaggle/input/preprocessing/train.csv')
test_df = pd.read_csv('/kaggle/input/preprocessing/test.csv')



# Preprocess reviews
train_df['processed_review'] = train_df['review'].apply(preprocess_text)
test_df['processed_review'] = test_df['review'].apply(preprocess_text)

# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Apply VADER to get sentiment scores for both datasets
train_df['vader_score'] = train_df['processed_review'].apply(lambda review: analyzer.polarity_scores(review)['compound'])
test_df['vader_score'] = test_df['processed_review'].apply(lambda review: analyzer.polarity_scores(review)['compound'])

# Prepare features and labels for the training set
X_train = train_df['processed_review']
y_train = train_df['sentiment']
X_test = test_df['processed_review']
y_test = test_df['sentiment']

# Vectorize the text
tfidf_vectorizer = TfidfVectorizer(max_features=50000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Add VADER scores as a feature
X_train_vader = train_df['vader_score'].values.reshape(-1, 1)
X_test_vader = test_df['vader_score'].values.reshape(-1, 1)

# Combine TF-IDF features with VADER scores
X_train_combined = np.hstack((X_train_tfidf.toarray(), X_train_vader))
X_test_combined = np.hstack((X_test_tfidf.toarray(), X_test_vader))

# Train the model on the training set
model = RandomForestClassifier()
model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = model.predict(X_test_combined)

# Evaluate and print the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy on test set:", accuracy)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Accuracy on test set: 0.8527
